In [193]:
import pandas as pd
import numpy as np

In [194]:
pd.options.mode.chained_assignment = None
data_path = '../data/'

In [195]:
# load data
results = pd.read_csv(data_path + 'epl_results_2022-23.csv')
teams = pd.read_csv('./../data/epl_clubs_info_2022-23.csv')

In [196]:
results.columns

Index(['Div', 'Date', 'Time', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR',
       'HTHG', 'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF',
       'HC', 'AC', 'HY', 'AY', 'HR', 'AR'],
      dtype='object')

In [197]:
results.head(1)

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,E0,05/08/2022,20:00,Crystal Palace,Arsenal,0,2,A,0,1,...,2,2,16,11,3,5,1,2,0,0


In [198]:
team_names = teams['club_name'].to_list()

## Create Goal Difference

The idea here is to get goal difference of each team after each game. This is to be plotted later. I will be plotting the GD of Pl big six (Manchester United, Liverpool, Arsenal, Chelsea, Manchester City and Tottenham).

In [199]:
def calculate_goal_diff(results: pd.DataFrame, team: str):
    """
    This function calculates the goal difference of each matches.

    Parameters
    ----------
    results: pd.DataFrame()
    team: str

    Returns
    -------
    gd: pd.DataFrame()
    """
    
    # get all matches played by the team
    matches = results.loc[(results['AwayTeam'] == team) | (results['HomeTeam'] == team)]
    matches['GD'] = 0 # create a new column called GD

    matches_played = matches.shape[0]

    # # calculate goal difference depending on team is home or away
    matches.loc[matches['HomeTeam'] == team, 'GD'] = (matches['FTHG'] - matches['FTAG'])
    matches.loc[matches['AwayTeam'] == team, 'GD'] = matches['FTAG'] - matches['FTHG']

    # add up the goal differnce after each match
    gd = pd.DataFrame(matches['GD'].cumsum())
    gd.reset_index(inplace=True)
    gd.drop(columns=['index'], inplace=True)
    gd.insert(0, 'MatchDay', range(matches_played))

    return gd

In [200]:
# save the goal difference of PL big six
big_six = ['Man United', 'Man City', 'Liverpool', 'Arsenal', 'Tottenham', 'Chelsea']

for team in big_six:
    gd = calculate_goal_diff(results, team)
    gd.to_csv(data_path + team + '_goal_diff.csv', index=False)

## Create Yellow and Red cards count for every teams

Here I want to get total yellow and red cards count, so that I can plot a bar chart to compare the counts of the big six.

In [201]:
def calculate_yellow_red(results: pd.DataFrame, team: str):
    """
    This function sums up the count of yelow and red cards of a given team from match results

    Parameters
    ----------
    results: pd.DataFrame()
    team: str

    Returns
    -------
    yellow_cards: int
    red_cards: int
    """
    # get all matches played by the team
    matches = results.loc[(results['AwayTeam'] == team) | (results['HomeTeam'] == team)]

    yellow_cards = 0
    red_cards = 0

    yellow_cards += matches.loc[matches['HomeTeam'] == team]['HY'].sum()
    yellow_cards += matches.loc[matches['AwayTeam'] == team]['AY'].sum()

    red_cards += matches.loc[matches['HomeTeam'] == team]['HR'].sum()
    red_cards += matches.loc[matches['AwayTeam'] == team]['AR'].sum()

    return yellow_cards, red_cards

In [202]:
data = []

for name in team_names:
    yellow_cards, red_cards = calculate_yellow_red(results=results, team=name)
    data.append({
        'team': name,
        'yellow_cards': yellow_cards,
        'red_cards': red_cards
    })

# convert data to Pandas dataframe and save as csv
y_r = pd.DataFrame(data)
y_r.to_csv(data_path + 'yellow_red.csv', index=False)

# Home and Away Goals
I want to make this a stacked bar chart where the home and away goals can be compared directly.

In [203]:
def calculate_goals(results: pd.DataFrame, team:str):
    """
    This function calculates both away and home goals of a given team from the given results
    
    Parameters
    ----------
    results: pd.DataFrame()
    team: str

    Returns
    -------
    away_goals: int
    home_goals: int
    """
    # get all matches played by the team
    matches = results.loc[(results['AwayTeam'] == team) | (results['HomeTeam'] == team)]

    away_goals = 0
    home_goals = 0

    away_goals += matches.loc[matches['AwayTeam'] == team]['FTAG'].sum()
    home_goals += matches.loc[matches['HomeTeam'] == team]['FTHG'].sum()

    return away_goals, home_goals


In [204]:
data = []

for name in team_names:
    away_goals, home_goals = calculate_goals(results=results, team=name)
    data.append({
        'team': name,
        'away_goals': away_goals,
        'home_goals': home_goals
    })

# convert data to Pandas dataframe and save as csv
goals = pd.DataFrame(data)
goals.to_csv(data_path + 'goals.csv', index=False)

# Create Team Movement
Create a df that shows the movement of every teams in the table after each matchday.

In [205]:
results['Date'] = pd.to_datetime(results['Date'], format="%d/%m/%Y")

In [206]:
team_results = []
total_match_days = 28

In [207]:
for team in team_names:
    matches = results.loc[(results['AwayTeam'] == team) | (results['HomeTeam'] == team)].reset_index()
    matches.drop(['index'], inplace=True, axis=1)
    team_results.append(matches.head(total_match_days))

In [208]:
standings = pd.DataFrame(np.nan, index=range(28), columns=team_names)

In [209]:
table = teams.iloc[:, :2]
table['GD'] = 0 # create a new column called GD
table['points'] = 0 # create a new column called Points
table['total_goals'] = 0 # create a new column called Total goals scored
table['matches_total'] = 0

In [210]:
for i in range(total_match_days):

    match_dict = {}
    for team in team_names:
        match_dict[team] = False

    for result in team_results:
        match = result.loc[i]

        if not match_dict[match['HomeTeam']]:
            table.loc[table['club_name'] == match['HomeTeam'], 'matches_total'] += 1
            table.loc[table['club_name'] == match['HomeTeam'], 'GD'] += (match['FTHG'] - match['FTAG'])
            table.loc[table['club_name'] == match['HomeTeam'], 'total_goals'] += (match['FTHG'])

            if match['FTR'] == 'H':
                table.loc[table['club_name'] == match['HomeTeam'], 'points'] += 3
            else:
                table.loc[table['club_name'] == match['HomeTeam'], 'points'] += 1

            match_dict[match['HomeTeam']] = True

        if not match_dict[match['AwayTeam']]:
            table.loc[table['club_name'] == match['AwayTeam'], 'matches_total'] += 1
            table.loc[table['club_name'] == match['AwayTeam'], 'GD'] += (match['FTAG'] - match['FTHG'])
            table.loc[table['club_name'] == match['AwayTeam'], 'total_goals'] += (match['FTAG'])

            if match['FTR'] == 'A':
                table.loc[table['club_name'] == match['AwayTeam'], 'points'] += 3
            else:
                table.loc[table['club_name'] == match['AwayTeam'], 'points'] += 1

            match_dict[match['AwayTeam']] = True
            
    table.sort_values(['points', 'GD', 'total_goals'], ascending=False, inplace=True)
    table.reset_index(inplace=True)
    table.drop(['index'], axis=1, inplace=True)

    for match in table.itertuples():
        standings.loc[i, match.club_name] = match.Index + 1


In [211]:
# save the standings as csv
standings.to_csv(data_path + 'standings.csv', index=False)